In [1]:
!pip install pandas
!pip install numpy

  Using cached pandas-2.2.0-cp310-cp310-macosx_11_0_arm64.whl (11.8 MB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [15]:
import pandas as pd
import numpy as np

np.random.seed(42)  

subjects = ['Mathematics', 'Physics', 'Chemistry']
topics_math = ['Algebra', 'Geometry', 'Calculus']
topics_physics = ['Mechanics', 'Thermodynamics', 'Optics']
topics_chemistry = ['Organic', 'Inorganic', 'Physical']

data_performance = []
for subject in subjects:
    topics = topics_math if subject == 'Mathematics' else topics_physics if subject == 'Physics' else topics_chemistry
    for topic in topics:
        accuracy = np.random.uniform(40, 90) 
        avg_response_time = np.random.uniform(30, 120)  
        data_performance.append([accuracy, avg_response_time, subject, topic])

df_performance = pd.DataFrame(data_performance, columns=['Accuracy', 'Average Response Time', 'Subject', 'Topic'])

data_content = [
    ['Mathematics', 'Algebra', 'Understanding variables and equations'],
    ['Mathematics', 'Geometry', 'Studying shapes, sizes, and the properties of space'],
    ['Mathematics', 'Calculus', 'Exploring changes and motion; limits and derivatives'],
    ['Physics', 'Mechanics', 'The study of motion and forces'],
    ['Physics', 'Thermodynamics', 'The study of heat and temperature'],
    ['Physics', 'Optics', 'The study of light and its properties'],
    ['Chemistry', 'Organic', 'Study of carbon compounds and life substances'],
    ['Chemistry', 'Inorganic', 'Study of inorganic compounds and materials'],
    ['Chemistry', 'Physical', 'Study of chemical thermodynamics and kinetics']
]

df_content = pd.DataFrame(data_content, columns=['Subject', 'Topic', 'Topic_Explanation'])

df_performance, df_content


(    Accuracy  Average Response Time      Subject           Topic
 0  42.542882              72.903077  Mathematics         Algebra
 1  55.582108             111.147132  Mathematics        Geometry
 2  64.751325              74.841130  Mathematics        Calculus
 3  85.592236             116.105219      Physics       Mechanics
 4  57.253298             114.621380      Physics  Thermodynamics
 5  51.833980              97.033214      Physics          Optics
 6  55.021901              98.461812    Chemistry         Organic
 7  86.794507             112.139767    Chemistry       Inorganic
 8  81.003568              81.890258    Chemistry        Physical,
        Subject           Topic  \
 0  Mathematics         Algebra   
 1  Mathematics        Geometry   
 2  Mathematics        Calculus   
 3      Physics       Mechanics   
 4      Physics  Thermodynamics   
 5      Physics          Optics   
 6    Chemistry         Organic   
 7    Chemistry       Inorganic   
 8    Chemistry        P

In [3]:
!pip install pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 41.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.3 kB/s eta 0:00:0000:0100:02
  Using cached annotated_types-0.6.0-py3-none-any.whl (12 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [16]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Performance(BaseModel):
    accuracy: float = Field(..., ge=0, le=100)
    average_response_time: float = Field(..., ge=0) 
    subject: str
    topic: str

class TopicExplanation(BaseModel):
    subject: str
    topic: str
    topic_explanation: str

performance_records = [Performance(accuracy=row.Accuracy, average_response_time=row['Average Response Time'],
                                   subject=row.Subject, topic=row.Topic) for index, row in df_performance.iterrows()]
content_records = [TopicExplanation(subject=row.Subject, topic=row.Topic, topic_explanation=row.Topic_Explanation)
                   for index, row in df_content.iterrows()]

performance_records[0], content_records[0]

(Performance(accuracy=42.54288186860654, average_response_time=72.90307746558943, subject='Mathematics', topic='Algebra'),
 TopicExplanation(subject='Mathematics', topic='Algebra', topic_explanation='Understanding variables and equations'))

In [20]:
from statistics import mean, stdev

def calculate_dynamic_thresholds(performance_data: List[Performance]) -> (float, float):
    accuracies = [p.accuracy for p in performance_data]
    response_times = [p.average_response_time for p in performance_data]
    
    mean_accuracy = mean(accuracies)
    stddev_accuracy = stdev(accuracies)
    mean_response_time = mean(response_times)
    stddev_response_time = stdev(response_times)
    
    accuracy_threshold = max(mean_accuracy - stddev_accuracy, 0) 
    time_threshold = mean_response_time + stddev_response_time
    
    return accuracy_threshold, time_threshold

def query_improvement_areas_dynamic(performance_data: List[Performance], content_data: List[TopicExplanation]) -> List[dict]:
    recommendations = []
    
    accuracy_threshold, time_threshold = calculate_dynamic_thresholds(performance_data)
    
    for performance in performance_data:
        if performance.accuracy < accuracy_threshold or performance.average_response_time > time_threshold:
            for content in content_data:
                if content.subject == performance.subject and content.topic == performance.topic:
                    recommendations.append({
                        'Subject': performance.subject,
                        'Topic': performance.topic,
                        'Current Accuracy': performance.accuracy,
                        'Current Average Response Time': performance.average_response_time,
                        'Suggested Study': content.topic_explanation
                    })
                    break  
    
    return recommendations

dynamic_improvement_recommendations = query_improvement_areas_dynamic(performance_records, content_records)
dynamic_improvement_recommendations


[{'Subject': 'Mathematics',
  'Topic': 'Algebra',
  'Current Accuracy': 42.54288186860654,
  'Current Average Response Time': 72.90307746558943,
  'Suggested Study': 'Understanding variables and equations'},
 {'Subject': 'Physics',
  'Topic': 'Mechanics',
  'Current Accuracy': 85.59223558776824,
  'Current Average Response Time': 116.10521889832368,
  'Suggested Study': 'The study of motion and forces'}]